In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyspark.pandas as ps
from py2neo import Graph
import sys, os

E:\Felpipe\Trabajo\Ciencias de datos en general\KaggleChallenges\venv\lib\site-packages\pyspark\pandas\__init__.py:43: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


# Eda "clásico" (tabular)

Este análisis es más fácil de realizar sobre PostgreSQL directamente. Así me puedo aprovechar de las ventajas de los DataFrames de Spark y su pacerido a Pandas.

In [2]:
PG_URL  = 'jdbc:postgresql://localhost:5432/graphs'
PG_USER = 'spark_ingest'
PG_PASS = 'GYleZAI2pTBKJYl9W1PL'
PG_SCHEMA = 'saml_d'
PG_TABLE1 =  'accounts'
PG_TABLE2 =  'transferences'
JDBC_JAR = r"C:\spark\spark-4.0.1-bin-hadoop3\jars\postgresql-42.7.4.jar"  
JDBC_BATCHSIZE = 10000
JDBC_FETCHSIZE = 10000
PYTHON = sys.executable  # python del kernel Jupyter

In [3]:
spark = (
    SparkSession.builder
    .appName("ieee-fraud-jupyter")
    .config("spark.jars", JDBC_JAR)
    .config("spark.driver.extraClassPath", JDBC_JAR)
    .config("spark.executor.extraClassPath", JDBC_JAR)
    .config("spark.sql.ansi.enabled", "false") #Para poder usar la API de pandas pues no soporta modo ansi
    .config("spark.pyspark.driver.python", PYTHON)
    .config("spark.pyspark.python", PYTHON)
    .config("spark.executorEnv.PYSPARK_PYTHON", PYTHON)
    .config("spark.sql.execution.arrow.pyspark.enabled", "false")
    .config("spark.driver.bindAddress", "127.0.0.1")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")  # Opcional: mejora performance
    .getOrCreate()
)

In [4]:
accounts_df = (
    spark.read.format("jdbc")
    .option("url", PG_URL)
    .option("dbtable", f"{PG_SCHEMA}.{PG_TABLE1}")  
    .option("user", PG_USER)
    .option("password", PG_PASS)
    .option("driver", "org.postgresql.Driver")
    .option("partitionColumn", "account")
    .option("lowerBound", "1")
    .option("upperBound", "10000000")
    .option("numPartitions", "2")
    .option("fetchsize", str(JDBC_FETCHSIZE))
    .load()
)

tx_df = (
    spark.read.format("jdbc")
    .option("url", PG_URL)
    .option("dbtable", f"{PG_SCHEMA}.{PG_TABLE2}")  
    .option("user", PG_USER)
    .option("password", PG_PASS)
    .option("driver", "org.postgresql.Driver")
    .option("partitionColumn", "id")
    .option("lowerBound", "1")
    .option("upperBound", "10000000")
    .option("numPartitions", "6")
    .option("fetchsize", str(JDBC_FETCHSIZE))
    .load()
)

In [5]:
accounts_dfps = accounts_df.pandas_api()
accounts_dfps.head()

,account,location
0,153883,UK
1,155774,UK
2,285416,UK
3,348111,UK
4,438458,UK


In [6]:
len(accounts_dfps)

855460

In [7]:
tx_dfps = tx_df.pandas_api()
tx_dfps.head()

,id,date_time,sender_account,receiver_account,amount,payment_currency,received_currency,payment_type,is_laundering,laundering_type
0,492016,2023-02-02 18:42:24,4946074624,7511880331,236.56,UK pounds,UK pounds,Cheque,0,Normal_Small_Fan_Out
1,492017,2023-02-05 11:35:14,2832588708,6966809819,15481.56,UK pounds,UK pounds,ACH,0,Normal_Small_Fan_Out
2,492018,2023-01-30 02:49:53,5970299678,1273751804,632.78,Dirham,UK pounds,Cross-border,0,Normal_Small_Fan_Out
3,492019,2023-01-21 01:55:49,8441492491,8402129113,4566.08,Euro,UK pounds,Cross-border,0,Normal_Fan_Out
4,492020,2022-12-28 18:28:21,5967297244,9554855591,1038.59,UK pounds,UK pounds,Cheque,0,Normal_Small_Fan_Out


In [8]:
tx_dfps.describe()

ConnectionRefusedError: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión